Right now, all pdfs have the section Interpretation and conclusions of economic evidence, so we aim to find that section. Since reference always follows this section, we stop at reference.

TODO:
For pdfs without section Interpretation and conclusions of economic evidence, we need NLP method to determine start and end.

In [27]:
import pdfplumber
import re
from fuzzywuzzy import process

def extract_section_from_pdf(pdf_path, target_section="Interpretation and conclusions of economic evidence", start_page=20):
    """
    Extracts the section with a name similar to 'target_section' from the given PDF, starting from 'start_page'.
    Works even if the section name is slightly different.
    """
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(start_page - 1, len(pdf.pages)):  # Adjust for 0-based index
            text.append(pdf.pages[page_num].extract_text())

    full_text = "\n".join(filter(None, text))  # Remove None values

    # Extract all section headers using a regex pattern for headings (e.g., "B.x.x Section Name")
    section_pattern = r"(B\.\d+\.\d+.*?)\n"
    section_matches = re.findall(section_pattern, full_text)

    if not section_matches:
        print("No section headers found.")
        return None

    # Find the best match for the target section
    best_match, score = process.extractOne(target_section, section_matches)

    if score < 80:  # Adjust threshold based on quality
        print(f"No close match found for '{target_section}'. Closest match: '{best_match}' (Score: {score})")
        return None

    print(f"Extracting section: {best_match}")

    # Find start position of matched section
    start_pos = full_text.find(best_match)

    # Find Reference to determine the end position
    next_match = re.search(r"B\.\d+\s+References", full_text[start_pos + len(best_match):])

    if next_match:
        end_pos = start_pos + len(best_match) + next_match.start()
        extracted_text = full_text[start_pos:end_pos]
    else:
        extracted_text = full_text[start_pos:]  # Extract till the end if no next header found

    return extracted_text

# Example usage:
pdf_path = "10 Typical Committee Papers/committee-papers-Baricitinib-not recommend.pdf"
extracted_text = extract_section_from_pdf(pdf_path, start_page=20) # Starts at page 20 because all titles are presented in the content section

if extracted_text:
    print(extracted_text)

Extracting section: B.3.15 Interpretation and conclusions of economic evidence
B.3.15 Interpretation and conclusions of economic evidence
Summary of cost-effectiveness evidence
The cost-effectiveness of baricitinib in severe AA was evaluated versus ‘watch and wait’, the
most clinically relevant comparator for this population, due to a lack of high-quality clinical data
for another comparator. In the probabilistic base case, baricitinib was associated with higher
costs (£***** per patient over a lifetime horizon) and higher benefits (**** QALYs per patient over
a lifetime horizon) compared with ‘watch and wait’. The base case probabilistic ICER was
£29,111 per QALY gained and did not differ meaningfully from the deterministic ICER (£29,395
per QALY gained). In absolute terms, base case probabilistic results suggested that baricitinib
4mg was associated with a total cost of £******, of which £****** related to drug acquisition. These
were partially compensated by cost savings due to redu

1. Extract pdf section
2. Run pdf section through NLP
3. Extract website section from website (committee)
4. Run website section using NLP
5. Use summarized website section to test accuracy of pdf section